# Capstone Project - Part 1: Data sourcing and processing
## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

#### Source and process the data then export to CSV to be used in Part 2. This is being done to keep the notebooks more legible and also stop me from having to re-run data generating code multiple times

In [1]:
import requests # URL handler
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup # HTMl Parser
from itertools import chain # usefullexploding lists

#### Using Beautiful Soup to extract the data and Requests to handle the URL. The code below simply iterates over the Tags generated by BS. I've saved down the final DataFrame as a CSV to use later (as indicated in the beggining, mostly to keep the notebooks cleaner)
#### In the final cell I've listed the DF shapes as requried by the questions - note that the increased size is expected given the original table had collapsed neighborhoods in the same Postal Code into one row

In [11]:
# load the URL, scrape the data and pass into a  datafram
wikipage = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(wikipage.text)
soup_table = soup.find_all("table")[0] # there is only 1 table so just take the first one

n_columns = 0
n_rows = 0
column_names = []

# use "soup" tags to retrieve the table data and construct data frame
for row in soup_table.find_all('tr'):

    td_tags = row.find_all('td')
    if len(td_tags) > 0:
        n_rows+=1
        if n_columns == 0:
            n_columns = len(td_tags)


    th_tags = row.find_all('th') 
    if len(th_tags) > 0 and len(column_names) == 0:
        for th in th_tags:
            column_names.append(th.get_text().strip())

columns = column_names if len(column_names) > 0 else range(0,n_columns)
df = pd.DataFrame(columns = columns,
                  index= range(0,n_rows))
row_marker = 0
for row in soup_table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        df.iat[row_marker,column_marker] = column.get_text()
        df.iat[row_marker,column_marker]=df.iat[row_marker,column_marker].strip()
        column_marker += 1
    if len(columns) > 0:
        row_marker += 1

# stip out Boroughs with "Not Assigned" and reset index
df_filtered = df[df['Borough']!="Not assigned"]
df_filtered.reset_index(drop=True, inplace=True)

# return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

# calculate lengths of splits
lens = df_filtered['Neighbourhood'].str.split(',').map(len)

# create new dataframe, repeating or chaining as appropriate - df_filtered_complete
df_fc = pd.DataFrame({'Postal Code': np.repeat(df_filtered['Postal Code'].str.strip(), lens),
                    'Borough': np.repeat(df_filtered['Borough'].str.strip(), lens),
                    'Neighbourhood': chainer(df_filtered['Neighbourhood'].str.strip())})


# final result is a table with 1 neighborbood per row, excluding those with no borough
df_fc.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Manor


In [3]:
df_fc.to_csv('Toronto_Neighborhoods_Cleaned.csv', sep = ',', header=df_fc.columns)

In [4]:
print('The dataframe has {} Boroughs with {} Postal Codes covering {} Neighborhoods.'.format(len(df_fc['Borough'].unique()),
                                                                                    len(df_fc['Postal Code'].unique()),
                                                                                    df_fc.shape[0]))

The dataframe has 10 Boroughs with 103 Postal Codes covering 217 Neighborhoods.
